# Mi primer proyecto
Volvemos a encontrarnos con el conjunto de datos de pingüinos PalmerPenguins. Esta vez trabajaremos con ellos desde Python, para ello instalaremos el paquete que nos permitirá cargarlo:

In [99]:
!pip install palmerpenguins


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


En segundo lugar, importaremos las librerías necesarias:

In [100]:
import pandas as pd
import numpy as np
from palmerpenguins import load_penguins

Ahora ya estamos en posición de empezar a trabajar con los datos.

1. Vamos a cargar el conjunto de datos. Muestra por pantalla el número de observaciones y sus características. Mira el tipo de datos de cada una de sus columnas.

In [101]:
penguins = load_penguins()
penguins
penguins.shape

(344, 8)

Como podemos observar tenemos 344 regiostros con 8 columnas o atributos cada uno de ellos.

In [102]:
penguins.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex', 'year'],
      dtype='object')

Ahora vamos a ver los tipos de los datos:

In [103]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 7   year               344 non-null    int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 21.6+ KB


Y una pequeña descripción:

In [104]:
penguins.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,342.000000,342.000000,342.000000,342.000000,344.000000
mean,43.921930,17.151170,200.915205,4201.754386,2008.029070
std,5.459584,1.974793,14.061714,801.954536,0.818356
min,32.100000,13.100000,172.000000,2700.000000,2007.000000
25%,39.225000,15.600000,190.000000,3550.000000,2007.000000
50%,44.450000,17.300000,197.000000,4050.000000,2008.000000
75%,48.500000,18.700000,213.000000,4750.000000,2009.000000
max,59.600000,21.500000,231.000000,6300.000000,2009.000000


2. Ya sabemos que este conjunto de datos tiene observaciones NA. Vamos a eliminarlas y a verificar que efectivamente no queda ninguno:



In [105]:
penguins.head(5).loc[3]

species                 Adelie
island               Torgersen
bill_length_mm             NaN
bill_depth_mm              NaN
flipper_length_mm          NaN
body_mass_g                NaN
sex                        NaN
year                      2007
Name: 3, dtype: object

Como podemos ver en este caso por ejemplo tenemos muchos valores NaN o "missing". Vamos a eliminar dichos valores para poder tener unos datos más limpios y tractables.

In [106]:
penguins_without_na = penguins.dropna()
penguins.columns[penguins.isna().any()]
penguins.notna().sum()
penguins.notnull().sum() # es equivalente a not na en pandas, pero no en numpy

species              344
island               344
bill_length_mm       342
bill_depth_mm        342
flipper_length_mm    342
body_mass_g          342
sex                  333
year                 344
dtype: int64

3. ¿Cuántos individuos hay de cada sexo? Puedes obtener la longitud media del pico según el sexo:

In [107]:
penguins_without_na.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex', 'year'],
      dtype='object')

Es casi seguro que debamos explorar la columna sex del data set

In [108]:
# penguins_without_na["sex"].head(5)
# penguins_without_na["sex"]
np.unique(penguins_without_na["sex"])

array(['female', 'male'], dtype=object)

Como podemos ver tenemos dos columnas diferentes de sex. Estas son 'male' y 'female'. Ahora vamos a contar cuantos individuos tenemos de cada uno de los tipos.

In [109]:
sex_col = penguins_without_na["sex"]
male_regs = penguins_without_na[sex_col == 'male']
total_male = male_regs.shape[0]
total_male

168

Como vemos tenemos 168 registros que son masculinos o 'male'. Si hacemos la diferencia con el total ya deberiamos obtener los femeninos o female.

In [110]:
penguins_without_na.shape[0]-total_male

165

Es decir que hay un total de 168 individuos femeninos. Podemos hecer lo mismo que con los masculinos para ver si el resultado es igual.

In [111]:
female_regs = penguins_without_na[sex_col == 'female']
female_regs.shape[0]

165

Como podemos ver, de las dos formas obtememos el mismo resultado, como era de esperar. Por tanto, tenemos un total de 168 registros masculinos y 156 femeninos.

Para obtener el tamaño medio del pico vamos a usar la función describe, que nos da mucha información sobre nuestro data set. Como los datos ya estan separados en el paso anterior solo hará falta aplicar las funciones sobre los datos filtrados.

In [112]:
#Calculamos la longitud media del pico en hombres
male_regs[male_regs.columns[2:4]].describe().iloc[1:2]

,bill_length_mm,bill_depth_mm
mean,45.854762,17.891071


In [113]:
#Calculamos la longitud media del pico en mujeres
female_regs[female_regs.columns[2:4]].describe().iloc[1:2]

,bill_length_mm,bill_depth_mm
mean,42.09697,16.425455


Como vemos la longitud media del pico es de 45 mm en hombres y 42 mm en mujeres. Por tanto los hombres suelen tener un pico mas largo que ellas.

4. Vamos a añadir una columna, vamos a realizar una estimación (muy grosera) del área del pico de los pingüinos (bill) tal como si esta fuese un rectángulo. Esta nueva columnas se llama bill_area y debe encontrarse en la última posición. Verifica que es correcto.

Para ello utilizaremos la función assign con la que crearemos la nueva columna que será el resultado del producto del bill_length_mm con bill_depth_mm.

In [114]:
penguins_b_area = penguins_without_na.assign(bill_area=penguins_without_na.bill_length_mm * penguins_without_na.bill_depth_mm)
penguins_b_area

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_area
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,731.17
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,687.30
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,725.40
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,708.31
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,809.58
...,...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,1104.84
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,787.35
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,902.72
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,965.20


Para hacer la verificación vamos a hacer el cálculo del área del primer registro [0] manualmente.

In [115]:
penguins_b_area.loc[0].bill_area

731.17

In [116]:
bill_length_depth_mm = penguins_without_na[penguins_without_na.columns[2:4]]
bill_length_depth_mm.loc[0].bill_length_mm * bill_length_depth_mm.loc[0].bill_depth_mm

731.17

5. Hagamos algo un poco más elaborado, vamos a realizar una agrupación en función del sexo y de la especie de cada observación. Queremos obtener solamente la información referente al sexo Femenino.

Pongamos que queremos obtener información sobre el 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm' y 'body_mass_g'. Para ello vamos a obtener un array con dichos nombres.

In [117]:
interest_columns = penguins_without_na.columns[2:6]
print(interest_columns)

Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], dtype='object')


Ahora que tenemos los datos de los quales vamos a extraer la información he pensado dos formas de hacerlo. La primera de ellas es la que me ha gustado más.

Esta trata de realizar las agrupaciones por partes. Primero de todo, agrupamos el dataset por 'sex'. Una vez tenemos el dataset agrupado obtenemos otro dataset filtrando por 'female'. Es decir, solo con registros del individuo femenino.

Una vez tenemos este dataset lo que hacemos es agrupar por especie y ya podemos obtener la información que necesitemos sobre los datos.

In [118]:
penguins_grouped_by_sex = penguins_without_na.groupby(["sex"])
female_penguins = penguins_without_na.loc[penguins_grouped_by_sex.groups['female']]

In [119]:
female_penguins_species_grouped = female_penguins.groupby(["species"])
female_penguins_species_grouped.describe()

bill_length_mm                                                   \
                   count       mean       std   min     25%   50%     75%   
species                                                                     
Adelie              73.0  37.257534  2.028883  32.1  35.900  37.0  38.800   
Chinstrap           34.0  46.573529  3.108669  40.9  45.425  46.3  47.375   
Gentoo              58.0  45.563793  2.051247  40.9  43.850  45.5  46.875   

                bill_depth_mm             ... body_mass_g          year  \
            max         count       mean  ...         75%     max count   
species                                   ...                             
Adelie     42.2          73.0  17.621918  ...     3550.00  3900.0  73.0   
Chinstrap  58.0          34.0  17.588235  ...     3693.75  4150.0  34.0   
Gentoo     50.5          58.0  14.237931  ...     4875.00  5200.0  58.0   

                                                                          
                  mean       std     min     25%     50%     75%     max  
species                                                                   
Adelie     2008.054795  0.814630  2007.0  2007.0  2008.0  2009.0  2009.0  
Chinstrap  2007.970588  0.869876  2007.0  2007.0  2008.0  2009.0  2009.0  
Gentoo     2008.068966  0.791669  2007.0  2007.0  2008.0  2009.0  2009.0  

[3 rows x 40 columns]

In [120]:
female_penguins_species_grouped[interest_columns].mean()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
species,,,,
Adelie,37.257534,17.621918,187.794521,3368.835616
Chinstrap,46.573529,17.588235,191.735294,3527.205882
Gentoo,45.563793,14.237931,212.706897,4679.741379


Vamos a la segunda opción. Se trata de hacer la agrupación conjunta, es decir, a la vez. Para ello necesitamos indicarle un parámetro nuevo que es **as_index** con el qual vamos a tratar la agrupación como si fuese una tabla nueva y cada uno de los miembros agrupados será tractado como una columna. Así será más facil realizar el filtrado.

Una vez se ha hecho la agrupación obtenemos un nuevo dataset que será el resultado de la operación que queremos realizar sobre los datos. En nuestro caso vamos a obtener la mediana aritmética de los atributos interesantes que hemos cogido al principio. 

In [121]:
penguins_grouped_by_sex_species = penguins_without_na.groupby(["sex", "species"], as_index=False)
mean_interested_columns_by_sex_species = penguins_grouped_by_sex_species[interest_columns].mean()
mean_interested_columns_by_sex_species

,sex,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,female,Adelie,37.257534,17.621918,187.794521,3368.835616
1,female,Chinstrap,46.573529,17.588235,191.735294,3527.205882
2,female,Gentoo,45.563793,14.237931,212.706897,4679.741379
3,male,Adelie,40.390411,19.072603,192.410959,4043.493151
4,male,Chinstrap,51.094118,19.252941,199.911765,3938.970588
5,male,Gentoo,49.473770,15.718033,221.540984,5484.836066


Como podemos observar hemos obtenido los datos correctamente pero aún tenemos los individuos masculinos.

Es, por tanto, en este punto que debemos realizar el filtrado.

In [122]:
sex_attr = mean_interested_columns_by_sex_species['sex']
mean_interested_columns_by_sex_species.loc[sex_attr == 'female']

,sex,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,female,Adelie,37.257534,17.621918,187.794521,3368.835616
1,female,Chinstrap,46.573529,17.588235,191.735294,3527.205882
2,female,Gentoo,45.563793,14.237931,212.706897,4679.741379


De esta manera, si comparamos los dos realutados veremos que son idénticos. Por tanto, se han visto dos formas de hacer lo mismo.

6. Como ya sabemos, la variable peso, se encuentra en gramos, la pasaremos a kg. Para ello crearemos una nueva columna llamada body_mass_kg y eliminaremos body_mass_g.

Para ello vamos a usar la función asign para crear la nueva columna llamada body_mass_g y drop para eliminar la columna antigua.

In [123]:
penguins_mass_kg = penguins_without_na.assign(body_mass_kg=penguins_without_na.body_mass_g / 1000)
penguins_mass_kg = penguins_mass_kg.drop(columns=["body_mass_g"])
penguins_mass_kg

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,sex,year,body_mass_kg
0,Adelie,Torgersen,39.1,18.7,181.0,male,2007,3.750
1,Adelie,Torgersen,39.5,17.4,186.0,female,2007,3.800
2,Adelie,Torgersen,40.3,18.0,195.0,female,2007,3.250
4,Adelie,Torgersen,36.7,19.3,193.0,female,2007,3.450
5,Adelie,Torgersen,39.3,20.6,190.0,male,2007,3.650
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,male,2009,4.000
340,Chinstrap,Dream,43.5,18.1,202.0,female,2009,3.400
341,Chinstrap,Dream,49.6,18.2,193.0,male,2009,3.775
342,Chinstrap,Dream,50.8,19.0,210.0,male,2009,4.100
